In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Making Custom NN using pytorch

In [8]:
import numpy as np
import pandas as pd

import torch

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Getting Dataset

In [10]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [12]:
df.shape

(569, 33)

In [14]:
df.drop(['Unnamed: 32', 'id'], axis=1, inplace=True)

# Data Splitting, Scaling and Label Encoding

In [18]:
train_x, test_x, train_y, test_y = train_test_split(df.drop(['diagnosis'], axis=1), df['diagnosis'], test_size=0.2, random_state=42)

scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

encoder = LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.transform(test_y)

# Conversion from numpy to pytorch

In [21]:
train_x = torch.from_numpy(train_x.astype(np.float32))
test_x = torch.from_numpy(test_x.astype(np.float32))
train_y = torch.from_numpy(train_y.astype(np.float32))
test_y = torch.from_numpy(test_y.astype(np.float32))

In [22]:
train_x.shape

torch.Size([455, 30])

# Model building

In [31]:
class MySimpleNN():
  def __init__(self, x):
    self.weight = torch.rand(x.shape[1], 1, dtype=torch.float32, requires_grad=True)
    self.bias = torch.rand(1, dtype=torch.float32, requires_grad=True)

  def forward(self, x):
    return torch.sigmoid(torch.matmul(x, self.weight) + self.bias)

  def loss_function(self, y_pred, y):
    # Clamp predictions to avoid log(0)
    epsilon = 1e-7
    y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)

    # Calculate loss
    loss = -(train_y * torch.log(y_pred) + (1 - train_y) * torch.log(1 - y_pred)).mean()
    return loss

# Important Parameter

In [32]:
leaning_rate = 0.1
epochs = 100

# Training Pipeline

In [33]:
model = MySimpleNN(train_x)

for epoch in range(epochs):
  # Foreword Propagation
  y_pred = model.forward(train_x)

  # Loss Calculation
  loss = model.loss_function(y_pred, train_y)

  # backword Pass
  loss.backward()


  # Parameter Updates
  with torch.no_grad():
    model.weight -= leaning_rate * model.weight.grad
    model.bias -= leaning_rate * model.bias.grad

  # Zero Gradient
  model.weight.grad.zero_()
  model.bias.grad.zero_()

  print(f'Epoch: {epoch+1}, Loss: {loss.item()}')

Epoch: 1, Loss: 3.571039915084839
Epoch: 2, Loss: 3.4399924278259277
Epoch: 3, Loss: 3.307166337966919
Epoch: 4, Loss: 3.17425799369812
Epoch: 5, Loss: 3.0364809036254883
Epoch: 6, Loss: 2.8943138122558594
Epoch: 7, Loss: 2.745234251022339
Epoch: 8, Loss: 2.585635185241699
Epoch: 9, Loss: 2.4228148460388184
Epoch: 10, Loss: 2.2554879188537598
Epoch: 11, Loss: 2.0898404121398926
Epoch: 12, Loss: 1.9256515502929688
Epoch: 13, Loss: 1.768455982208252
Epoch: 14, Loss: 1.617266058921814
Epoch: 15, Loss: 1.469307780265808
Epoch: 16, Loss: 1.3361629247665405
Epoch: 17, Loss: 1.215718388557434
Epoch: 18, Loss: 1.1143109798431396
Epoch: 19, Loss: 1.0319530963897705
Epoch: 20, Loss: 0.9672772288322449
Epoch: 21, Loss: 0.9180248379707336
Epoch: 22, Loss: 0.8813595771789551
Epoch: 23, Loss: 0.8543918132781982
Epoch: 24, Loss: 0.8345235586166382
Epoch: 25, Loss: 0.8196316957473755
Epoch: 26, Loss: 0.8081129789352417
Epoch: 27, Loss: 0.7988329529762268
Epoch: 28, Loss: 0.791031002998352
Epoch: 29, L

# Model Evaluation

In [35]:
with torch.no_grad():
  y_pred = model.forward(test_x)
  y_pred = (y_pred > 0.9).float()
  accuracy = (y_pred == test_y).float().mean()
  print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.6228070259094238
